## Init and import

This depends on a new custom module in concordia
Run "pip install --editable .[dev]" in your environment to install the new build if you haven't

In [1]:
# @title Imports
import sys
import os
path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, path)
import datetime

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('llama')

In [14]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

In [37]:
# from concordia.language_model import huggingface_model
import llama_cpp_model
import importlib
importlib.reload(llama_cpp_model)

model = llama_cpp_model.LlamaCppLanguageModel(
    repo_id="TheBloke/Llama-2-7B-Chat-GGUF",
    filename="llama-2-7b-chat.Q4_K_S.gguf",
    logits=True,
    verbose=True,
)

# model = llama_cpp_model.LlamaCppLanguageModel(
#     repo_id="TheBloke/Llama-2-70B-Chat-GGUF",
#     filename="llama-2-70b-chat.Q4_K_M.gguf",
#     logits=True,
#     verbose=True,
# )

In [ ]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [35]:
measurements = measurements_lib.Measurements()

In [38]:
import custom_components.logits_query_metric as lm
import importlib
importlib.reload(lm)

context = """<<SYS>>
You are a helpful, respectful and honest assistant.
<</SYS>>

If the following statement is true, answer "yes". If it is false, answer "no". Only answer with a single word.
"""

quiz_metric = lm.LogitsQueryMetric(model, 
                                   "Alice", 
                                   clock, 
                                   "./custom_components/truth_or_lie.json", 
                                   measurements=measurements, 
                                   verbose=True,
                                   query=["yes", "no"]) # query should match up with order of options in the question json
quiz_metric.observe(context)

The LLM response is: 
  Yes
<0x0A>: 11.853142738342285
1: 7.877666473388672
▁*: 7.622175216674805
<0x09>: 7.3231425285339355
2: 7.2437520027160645
Logit for yes is -0.051347583532333374
Logit for no is 0.9764198064804077
To the question "1 + 1 = 2", the agent answered "no", the probs are {'yes': 0.2635171710679263, 'no': 0.7364828289320737}
The LLM response is: 
  Yes
▁a: 18.427715301513672
▁blue: 17.08873176574707
▁the: 15.907150268554688
▁green: 15.344120979309082
▁to: 14.865976333618164
Logit for yes is 1.6525318622589111
Logit for no is 3.219536304473877
To the question "Red is a color", the agent answered "no", the probs are {'yes': 0.17264385158039827, 'no': 0.8273561484196018}
The LLM response is: 
  No
▁is: 18.323223114013672
': 15.17531967163086
qu: 14.21264362335205
lings: 12.216053009033203
▁has: 12.069131851196289
Logit for yes is 1.0699762105941772
Logit for no is 2.635185480117798
To the question "Earth is flat", the agent answered "no", the probs are {'yes': 0.1729004210

In [5]:
import importlib
import examples.custom_components.logits_query_metric as lm
importlib.reload(lm)

results = quiz_metric.get_results()
import csv
# Create a csv file with the results if not already created
if not os.path.exists('./results/'):
    os.makedirs('./results/')
with open('./results/logits.txt', 'w') as file:
    # Write Question: Response, and space after each pair
    writer = csv.writer(file)
    for result in results:
        writer.writerow(result)
    file.close()

NameError: name 'quiz_metric' is not defined